In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import display, HTML
from ipywidgets import interact, FloatSlider, IntSlider, Button, Output 
display(HTML("<style>.rendered_html.text_cell_render {max-width:600px; }</style>")) 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Helper

## Lex Fridman on Youtube

In [ ]:
import re
from functools import reduce
lex_episode_rx = re.compile(r"(?P<guests>.+):.+\|.+\#(?P<episode>[0-9]+)")

def split_guests(string):
    gs = string.split(",")
    gs = [g.split("&") for g in gs]
    gs = reduce(lambda a,b: a + b, gs)
    gs = [g.strip() for g in gs]
    return gs

def extract_info(title):
    m = lex_episode_rx.match(v["title"])
    if m is not None:
        d = m.groupdict()
        d["guests"] = split_guests(d["guests"])
        return d
    else: 
        return None
    
def extract_ep(title):
    info = extract_info(title)
    if info is not None: return info["episode"]
    else: return None

# Loading audio data from youtube

Resourses and useful links:

 - Thanks Andrej Karpathy: https://karpathy.ai/lexicap/

## Grab youtube links
Let's get all the videos from a given playlist. 
- Code snippet from the examples on https://pypi.org/project/youtube-search-python/.

In [ ]:
from youtubesearchpython import Playlist, playlist_from_channel_id

playlist = Playlist("https://www.youtube.com/playlist?list=PLrAXtmErZgOdP_8GztsuKi9nrraNbKKp4")

print(f'Videos Retrieved: {len(playlist.videos)}')

while playlist.hasMoreVideos:
    print('Getting more videos...')
    playlist.getNextVideos()
    print(f'Videos Retrieved: {len(playlist.videos)}')

print('Found all the videos.')
vids = playlist.videos

Videos Retrieved: 100
Getting more videos...
Videos Retrieved: 200
Getting more videos...
Videos Retrieved: 300
Getting more videos...
Videos Retrieved: 325
Found all the videos.


In [ ]:
v = vids[0]
print(v.keys())

dict_keys(['id', 'thumbnails', 'title', 'channel', 'duration', 'accessibility', 'link', 'isPlayable'])


## Grab audio data from youtube
Grab the audio data from youtube. See the CLI examples at https://github.com/yt-dlp/yt-dlp.

In [ ]:
from pathlib import Path
from cgnai.utils import cgnai_home
from cgnai.fileio import dump

target_dir = cgnai_home()/"local/data/Lex_Fridman_Podcast/Examples"

i = 2
v     = vids[i]
info  = extract_info(v["title"])
vid   = v["id"]
fname = target_dir/f"lex_#{info['episode']}.mp3"
info = extract_info(v["title"])
info["fname"] = fname.name
info["id"] = vid

dump(info, f"{fname.name}_info")
info

{'guests': ['John Danaher'],
 'episode': '328',
 'fname': 'lex_#328.mp3',
 'id': 'iZRbD7q1n-U'}

In [ ]:
!yt-dlp -x --audio-format mp3 -o $fname -- $vid

[youtube] iZRbD7q1n-U: Downloading webpage
[youtube] iZRbD7q1n-U: Downloading android player API JSON
[info] iZRbD7q1n-U: Downloading 1 format(s): 251
[download] Destination: /Users/mirko/Workspace/cgn.ai/cgnai/local/data/Lex_Fridman_Podcast/Examples/lex_#328.webm
[download] 100% of  230.99MiB in 00:02:03 at 1.86MiB/s0;33m00:00
[ExtractAudio] Destination: /Users/mirko/Workspace/cgn.ai/cgnai/local/data/Lex_Fridman_Podcast/Examples/lex_#328.mp3
Deleting original file /Users/mirko/Workspace/cgn.ai/cgnai/local/data/Lex_Fridman_Podcast/Examples/lex_#328.webm (pass -k to keep)
